In [1]:
from math import isnan

import pandas as pd

In [2]:
DATA_ROOT = "../exascale_data"

In [3]:
df = pd.read_csv(f"{DATA_ROOT}/prerelease/node_tables/Ath_master_annotation_v01.txt", sep="\t")
df.tail()

,GID,defline,symbols,names,KO_effect,GO,GOdesc,mapman_code,mapman_name,mapman_desc
39814,AT4G09945,NaN,NaN,NaN,NaN,NaN,NaN,35.2,not assigned.unknown,long noncoding RNA
39815,AT4G09955,NaN,NaN,NaN,NaN,NaN,NaN,35.2,not assigned.unknown,Natural antisense transcript overlaps with AT4...
39816,AT4G09975,NaN,NaN,NaN,NaN,NaN,NaN,35.2,not assigned.unknown,Natural antisense transcript overlaps with AT4...
39817,AT4G09985,NaN,NaN,NaN,NaN,NaN,NaN,35.2,not assigned.unknown,Natural antisense transcript overlaps with AT4...
39818,AT4G09995,NaN,NaN,NaN,NaN,NaN,NaN,35.2,not assigned.unknown,long noncoding RNA


In [4]:
isinstance(float('nan'), str) and float("asdf")

False

In [5]:
set([
    (len(go.split("|")) == len(godesc.split("|")))
    if isinstance(go, str) else go
    for go, godesc in df[["GO", "GOdesc"]].values
    if not (isinstance(go, float) and isnan(float(go)))
])

{True}

In [6]:
%%time
godict = dict()
gotuples = sorted(sum([
    tuple(zip(go.split("|"), godesc.split("|")))
    for go, godesc in df[["GO", "GOdesc"]].values
    if not (isinstance(go, float) and isnan(float(go)))
], ()))
for go_, godesc_ in gotuples:
    if go_ not in godict:
        godict[go_] = godesc_
        continue
    assert godict[go_] == godesc_

CPU times: user 17.5 s, sys: 45.4 ms, total: 17.6 s
Wall time: 17.5 s


In [7]:
tuple(godict.items())[:5]

(('GO:0000002', 'mitochondrial genome maintenance'),
 ('GO:0000003', 'reproduction'),
 ('GO:0000009', 'alpha-1,6-mannosyltransferase activity'),
 ('GO:0000012', 'single strand break repair'),
 ('GO:0000014', 'single-stranded DNA endodeoxyribonuclease activity'))

In [8]:
isinstance(float('nan'), float)

True